# Installation

In [1]:
!pip install -q --upgrade keras-cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 16.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/NoLaTeX

/content/drive/MyDrive/Colab Notebooks/NoLaTeX


In [6]:
!pwd

/content/drive/MyDrive/Colab Notebooks/NoLaTeX


In [ ]:
!pip install -e .

Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/NoLaTeX
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 557.1/578.0 MB 2.2 MB/s eta 0:00:10

In [ ]:
#uncoment to have time displayed on every cell

# !pip install ipython-autotime
# %load_ext autotime

# Imports

In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
#import tensorflow as tf
#import functions
from keras.utils import load_img
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.models import load_model

from keras_cv import visualization
from keras_cv.layers import Resizing, NonMaxSuppression
from keras_cv.callbacks import PyCOCOCallback
from keras_cv.models import YOLOV8Detector

from nolatex.ml_logic.utils import load_dataset

2024-03-15 16:40:39.532025: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 16:40:39.533321: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 16:40:39.543642: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 16:40:39.618273: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 16:40:41.115233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

# Define Paths

In [2]:
json_path = "/home/diegoberan/code/ChilleeX/NoLaTeX/initial_test_data/batch_1_sample5/kaggle_data_1.json"
img_dir = "/home/diegoberan/code/ChilleeX/NoLaTeX/initial_test_data/batch_1_sample5/images"

# Load Data into the right format

In [3]:
dataset, class_mapping, low_contrast_imgs = load_dataset(img_dir=img_dir, json_path=json_path)

num_classes = len(class_mapping)

In [4]:
dataset["images"][0].shape

TensorShape([342, None, None])

In [ ]:
from sys import getsizeof
getsizeof(dataset)

# Image Resizing

In [5]:
inference_resizing = Resizing(
    640, 640, pad_to_aspect_ratio=True, bounding_box_format="xywh"
)

In [6]:
dataset_resized = inference_resizing(dataset)

# Starting the model

In [8]:
model_dir = ""
model_name = "name" + ".keras"
model_path = os.path.join(model_dir, model_name)

#Set model Params here
backbone_model = "resnet50_imagenet"
#Optimizer Params
base_lr = 0.005
momentum = 0.9
global_clipnorm = 10.0
#Loss Params
classification_loss = "binary_crossentropy"
box_loss = "ciou"
#Train Params
number_epochs = 1
batch_size = 32

In [9]:
model = YOLOV8Detector.from_preset(
    backbone_model,
    bounding_box_format="xywh",
    num_classes=num_classes,
)

In [10]:
# including a global_clipnorm is extremely important in object detection tasks
optimizer = SGD(
    learning_rate=base_lr, momentum=momentum, global_clipnorm=global_clipnorm
)

In [11]:
model.compile(
    classification_loss=classification_loss,
    box_loss=box_loss,
    optimizer=optimizer,
)

In [ ]:
#Needs to be tested
# coco_metrics_callback = PyCOCOCallback(
#     resized_data, bounding_box_format="xywh"
# )

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = os.path.join(model_dir, f"{model_name}-cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

# Calculate the number of batches per epoch
n_batches = len(dataset['images']) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch"
    #save_freq=5*n_batches
    )

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [12]:
#by just following the example notebook I managed to train 1 epoch in 45 minutes
#did not manage to run fit with 16 images, colab breaks when reaching around the 3 min mark
model.fit(
    dataset_resized,
    # Run for 10-35~ epochs to achieve good scores.
    epochs=number_epochs,
    #batch_size=batch_size,
    #callbacks=[cp_callback]
    #callbacks=[coco_metrics_callback]
)

#model.save(model_path)

ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv1_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 640, 640, 1)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 640, 640, 1), dtype=float32)
  • training=True
  • mask=None

: 

# Loading Models

In [ ]:
# Restore the weights

#Important model needs to be initialized first
model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
# Loading a complete model

new_model = load_model('my_model.keras')

# Ploting predict

In [ ]:
test_image_path = "/content/drive/MyDrive/Pitch/test data/example_img.jpg"
image_test = load_img(test_image_path)
image_test = np.array(image_test)
image_test_resized = inference_resizing([image_test])

In [ ]:
#Simple visualization (Already tested)
y_pred = model.predict(image_test_resized)
# y_pred is a bounding box Tensor:
# {"classes": ..., boxes": ...}
visualization.plot_bounding_box_gallery(
    image_test_resized,
    value_range=(0, 255),
    rows=1,
    cols=1,
    y_pred=y_pred,
    scale=5,
    font_scale=0.7,
    bounding_box_format="xywh",
    class_mapping=class_mapping,
)

In [ ]:
#Needs to be tested
model.prediction_decoder = NonMaxSuppression(
    bounding_box_format="xywh",
    from_logits=True,
    iou_threshold=0.5,
    confidence_threshold=0.75,
)
visualize_detections(model, dataset=visualization_ds, bounding_box_format="xywh")

# visualization_ds on example notebook is en evaluation set
# visualization_ds = eval_ds.unbatch()
# visualization_ds = visualization_ds.ragged_batch(16)
# visualization_ds = visualization_ds.shuffle(8)

#Experimental Stuff


In [ ]:
class VisualizeDetections(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        visualize_detections(
            self.model, bounding_box_format="xywh", dataset=visualization_ds
        )
